# Week-5: The Battle of Neighborhoods

## Data Scraping

### In this segment, I am scrapping the Canada neighborhood data from Wikipedia page and merging with GeoCode to use it later with Foursquare

In [2]:
# Importing libraries
import requests
import pandas as pd
import numpy as np

In [3]:
# Scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')

Page download successful


In [4]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
df_html.shape

(180, 3)

In [6]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [7]:
df_html.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]

In [9]:
n_empty_neighborhood

0

In [10]:
#Group by Postcode / Borough
df_postcodes = df_html.groupby(['Postal Code','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df_postcodes.shape

(103, 3)

In [12]:
#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# Make sure both dataframes have the same 
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_postcodes.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [14]:
# Merge both datasets
df_Toronto_coordinates = pd.merge(df_postcodes, df_coordinates, on='PostalCode')
df_Toronto_coordinates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
#Export to .CSV
df_Toronto_coordinates.to_csv('Toronto_Coordinates.csv')

In [16]:
pip install python-forge

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [18]:
from sklearn.cluster import KMeans
import folium 

In [20]:
#count Bourough and Neighborhood
df_Toronto_coordinates.groupby('Borough').count()['Neighbourhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighbourhood, dtype: int64

In [21]:
df_toronto = df_Toronto_coordinates[df_Toronto_coordinates['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/Users/shrutinanda/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
df_toronto.shape

(39, 5)

In [24]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighbourhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighbourhood, dtype: int64


In [25]:

#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [26]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [27]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

# Map of Toronto Neighborhoods using Folium

In [29]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighbourhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [30]:
CLIENT_ID = 'SREQU3ACETQ4F5HI5R3CVWTVTQTK1ELYCQ5XVG0YDJQLKSW5' # your Foursquare ID. Kept blank for Purpose, was executed with values
CLIENT_SECRET = 'PDT4UVI1RDMYMZS2DZSQBHFGNHZ40NZFFT04LOKUOWLN1VNE' # your Foursquare Secret, Kept blank for Purpose, was executed with values
VERSION = '20201128' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

### Four Square API to pull the data related to Venue

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [37]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [38]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [39]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Restaurant', 'Juice Bar',
       'Pizza Place', 'Bookstore', 'Furniture / Home Store',
       'Dessert Shop', 'Bubble Tea Shop', 'Grocery Store', 'Spa',
       'Coffee Shop', 'Bakery', 'Caribbean Restaurant',
       'Indian Restaurant', 'Café', 'Lounge', 'Frozen Yogurt Shop',
       'Sushi Restaurant', 'American Restaurant', 'Liquor Store', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Fish Market', 'Gay Bar', 'Seafood Restaurant', 'Cheese Shop',
       'Middle Eastern Restaurant', 'Thai Restaurant', 'Stationery Store',
       'Comfort Food Restaurant', 'Coworking Space',
       'Latin American Restaurant', 'Gastropub', 'Clothing Store',
       'Gym / Fitness Cent

In [40]:
# check if the results contain "Thai Restaurants"
#please note I changed the data to Thai because I was previously writing the code using Asian but the number is so small
"Indian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [41]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighbourhoods'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1639, 241)


,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
to_grouped = to_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 241)


,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.058824,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.014706,0.000000,0.000000,0.014706,0.00,0.014706
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.013333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,0.000000,...,0.013333,0.000000,0.000000,0.00,0.000000,0.000000,0.013333,0.000000,0.00,0.026667
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [43]:
len(to_grouped[to_grouped["Indian Restaurant"] > 0])

7

In [44]:
to_Indian = to_grouped[["Neighbourhoods","Indian Restaurant"]]

In [45]:
to_Indian.shape

(39, 2)

# Cluster Neighborhoods

## Run K-Means to cluster the neighborhoods in Toronto into 4 clusters!!!

In [46]:
# Set number of clusters
toclusters = 4

to_clustering = to_Indian.drop(["Neighbourhoods"], 1)

# Run K-Means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 3, 0, 1, 0], dtype=int32)

In [47]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_Indian.copy()

# Add Clustering Labels
to_merged["Cluster Labels"] = kmeans.labels_

In [48]:

to_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
to_merged.head()


,Neighbourhood,Indian Restaurant,Cluster Labels
0,Berczy Park,0.000000,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.014706,3


In [49]:
# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighbourhood"), on="Neighbourhood")

print(to_merged.shape)
to_merged.head()

(1639, 9)


,Neighbourhood,Indian Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,43.644771,-79.373306,Goose Island Brewhouse,43.647329,-79.373541,Beer Bar


In [50]:
# Sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1639, 9)


,Neighbourhood,Indian Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.000000,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Lobby Lounge at the Shangri-La Toronto,43.649155,-79.386546,Lounge
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Soho House Toronto,43.648734,-79.386541,Speakeasy
25,"Richmond, Adelaide, King",0.000000,0,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
...,...,...,...,...,...,...,...,...,...
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,3,43.640816,-79.381752,Ontario Square,43.639378,-79.382096,Plaza
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,3,43.640816,-79.381752,Le Germain Hotel,43.643125,-79.380918,Hotel
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,3,43.640816,-79.381752,Sharetea,43.640176,-79.379606,Bubble Tea Shop
4,Central Bay Street,0.014706,3,43.657952,-79.387383,Lettieri Espresso Bar + Cafe,43.659185,-79.387264,Café


# Visualizing and Plotting the Clusters

In [51]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [52]:
# Create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# Set color scheme for the Clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighbourhood Latitude'], to_merged['Neighbourhood Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Save the map as HTML file

In [53]:
map_clusters.save('map_clusters.html')

# Studying Clusters Individually

## Cluster-0

In [54]:
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighbourhood,Indian Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Lobby Lounge at the Shangri-La Toronto,43.649155,-79.386546,Lounge
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Soho House Toronto,43.648734,-79.386541,Speakeasy
25,"Richmond, Adelaide, King",0.0,0,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
...,...,...,...,...,...,...,...,...,...
11,"First Canadian Place, Underground city",0.0,0,43.648429,-79.382280,Ruth's Chris Steak House,43.649917,-79.385724,Steakhouse
12,"Forest Hill North & West, Forest Hill Road Park",0.0,0,43.696948,-79.411307,Forest Hill Road Park,43.697945,-79.406605,Park
12,"Forest Hill North & West, Forest Hill Road Park",0.0,0,43.696948,-79.411307,Nikko Sushi Japenese Restaurant,43.700443,-79.407957,Sushi Restaurant
12,"Forest Hill North & West, Forest Hill Road Park",0.0,0,43.696948,-79.411307,Chimichanga,43.700354,-79.407605,Mexican Restaurant


## Cluster-1

In [57]:
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighbourhood,Indian Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Factory Girl,43.676693,-79.356299,American Restaurant
36,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Book City,43.677413,-79.352734,Bookstore
36,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Bar Oak,43.677931,-79.348724,Lounge
36,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Menchie's,43.678309,-79.348105,Frozen Yogurt Shop
36,"The Danforth West, Riverdale",0.022727,1,43.679557,-79.352188,Second Cup,43.677232,-79.352898,Coffee Shop
...,...,...,...,...,...,...,...,...,...
30,"St. James Town, Cabbagetown",0.020000,1,43.667967,-79.367675,Riverdale Farmer's Market,43.666891,-79.362503,Farmers Market
30,"St. James Town, Cabbagetown",0.020000,1,43.667967,-79.367675,Hey Lucy,43.664075,-79.368655,Italian Restaurant
30,"St. James Town, Cabbagetown",0.020000,1,43.667967,-79.367675,Jetfuel Coffee,43.665295,-79.368335,Coffee Shop
30,"St. James Town, Cabbagetown",0.020000,1,43.667967,-79.367675,Sunny Green Vegetable and Fruit,43.667555,-79.372435,Market


## Cluster-2

In [58]:
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighbourhood,Indian Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Creeds Coffee Bar,43.674100,-79.410838,Coffee Shop
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Fet Zun,43.675147,-79.406346,Middle Eastern Restaurant
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Big Crow,43.675896,-79.403680,BBQ Joint
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Jean Sibelius Square,43.671426,-79.408831,Park
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Madame Boeuf And Flea,43.675240,-79.406620,Burger Joint
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Dish Cooking Studio,43.674066,-79.410764,Café
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Shoppers Drug Mart,43.674959,-79.407986,Pharmacy
34,"The Annex, North Midtown, Yorkville",0.052632,2,43.67271,-79.405678,Toronto Archives,43.676447,-79.407509,History Museum


## Cluster-3

In [59]:
to_merged.loc[to_merged['Cluster Labels'] == 3]

,Neighbourhood,Indian Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,Central Bay Street,0.014706,3,43.657952,-79.387383,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
4,Central Bay Street,0.014706,3,43.657952,-79.387383,Mercatto,43.660391,-79.387664,Italian Restaurant
4,Central Bay Street,0.014706,3,43.657952,-79.387383,The Elm Tree Restaurant,43.657397,-79.383761,Modern European Restaurant
4,Central Bay Street,0.014706,3,43.657952,-79.387383,Hailed Coffee,43.658833,-79.383684,Coffee Shop
4,Central Bay Street,0.014706,3,43.657952,-79.387383,Neo Coffee Bar,43.660140,-79.385870,Coffee Shop
...,...,...,...,...,...,...,...,...,...
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,3,43.640816,-79.381752,Ontario Square,43.639378,-79.382096,Plaza
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,3,43.640816,-79.381752,Le Germain Hotel,43.643125,-79.380918,Hotel
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,3,43.640816,-79.381752,Sharetea,43.640176,-79.379606,Bubble Tea Shop
4,Central Bay Street,0.014706,3,43.657952,-79.387383,Lettieri Espresso Bar + Cafe,43.659185,-79.387264,Café


## Conclusion

There are no Indian Restaurant in Cluster 1, which includes neighborhood like `Richmond, Adelaide, King, Parkdale, Queen's park, Forest Hill, etc`. 
Cluster 2 and 3 have good count of Indian Restaurants, whereas Cluster 0 has some restaurants in that neighborhood. 

Studying the above cluster, I would recommend the food chain restaurant to open an Indian restaurant in Cluster 1 with little to no competition. 